In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
SUBMISSION_TYPE = 'validation'

In [3]:
top_forecast = pd.read_csv('../data/refined/forecast_prophet_store_dept.csv')
top_forecast = top_forecast[top_forecast.y.isnull()]
top_forecast['store_id'] = top_forecast['id'].str[:4]
top_forecast['dept_id'] = top_forecast['id'].str[5:]
top_forecast.head()

,id,ds,d,y,yhat,store_id,dept_id
1913,CA_1_FOODS_1,2016-04-25,1914,NaN,246,CA_1,FOODS_1
1914,CA_1_FOODS_1,2016-04-26,1915,NaN,229,CA_1,FOODS_1
1915,CA_1_FOODS_1,2016-04-27,1916,NaN,222,CA_1,FOODS_1
1916,CA_1_FOODS_1,2016-04-28,1917,NaN,239,CA_1,FOODS_1
1917,CA_1_FOODS_1,2016-04-29,1918,NaN,303,CA_1,FOODS_1


In [4]:
top_down_ratio = pd.read_csv('../data/refined/top_down_ratio_' + SUBMISSION_TYPE + '.csv')
top_down_ratio.head()

,id,dept_id,store_id,d,sales_ratio
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1,CA_1,1914,0.001364
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1,CA_1,1914,0.000957
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1,CA_1,1914,0.000699
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1,CA_1,1914,0.003646
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1,CA_1,1914,0.002837


In [5]:
top_down_ratio.describe()[['sales_ratio']]

,sales_ratio
count,853720.000000
mean,0.002319
std,0.003365
min,0.000079
25%,0.000685
50%,0.001337
75%,0.002535
max,0.176220


In [6]:
top_down_ratio.groupby(['dept_id', 'store_id', 'd'])['sales_ratio'].sum().reset_index()

,dept_id,store_id,d,sales_ratio
0,FOODS_1,CA_1,1914,0.957328
1,FOODS_1,CA_1,1915,0.964345
2,FOODS_1,CA_1,1916,0.960855
3,FOODS_1,CA_1,1917,0.974122
4,FOODS_1,CA_1,1918,0.980753
...,...,...,...,...
1955,HOUSEHOLD_2,WI_3,1937,1.013891
1956,HOUSEHOLD_2,WI_3,1938,1.014499
1957,HOUSEHOLD_2,WI_3,1939,1.014573
1958,HOUSEHOLD_2,WI_3,1940,1.013389


In [20]:
from scipy.special import softmax

top_down_ratio['sales_ratio_softmax'] = test \
    .groupby(['dept_id', 'store_id', 'd'])['sales_ratio'] \
    .apply(lambda x: softmax(x))

top_down_ratio['sales_ratio_rescale'] = test \
    .groupby(['dept_id', 'store_id', 'd'])['sales_ratio'] \
    .apply(lambda x: x / x.sum())

In [ ]:
---------

In [34]:
forecast_df = pd.merge(top_down_ratio, top_forecast.drop(columns='id'))

In [35]:
forecast = np.array(forecast_df['sales_ratio_softmax'] * forecast_df['yhat'])

In [36]:
output_id = forecast_df.id.unique()
forecast_df = pd.DataFrame(forecast.reshape(output_id.shape[0], 28))
forecast_df.columns = ['F' + str(int(c) + 1) for c in forecast_df.columns]
forecast_df['id'] = output_id
if SUBMISSION_TYPE == 'validation':
    forecast_df['id'] = forecast_df['id'].str.replace('evaluation', 'validation')
forecast_df.set_index('id', inplace=True)

In [37]:
res = pd.read_csv('../data/raw/sample_submission.csv')
res.set_index('id', inplace=True)
res.update(forecast_df)
res.reset_index(inplace=True)

In [45]:
top_down_ratio

,id,dept_id,store_id,d,sales_ratio,sales_ratio_softmax,sales_ratio_rescale
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1,CA_1,1914,0.001364,0.002401,0.001354
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1,CA_1,1914,0.000957,0.002400,0.000950
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1,CA_1,1914,0.000699,0.002400,0.000693
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1,CA_1,1914,0.003646,0.002407,0.003618
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1,CA_1,1914,0.002837,0.002405,0.002815
...,...,...,...,...,...,...,...
853715,FOODS_3_823_WI_3_evaluation,FOODS_3,WI_3,1941,0.000326,0.001214,0.000317
853716,FOODS_3_824_WI_3_evaluation,FOODS_3,WI_3,1941,0.000209,0.001214,0.000203
853717,FOODS_3_825_WI_3_evaluation,FOODS_3,WI_3,1941,0.000491,0.001214,0.000478
853718,FOODS_3_826_WI_3_evaluation,FOODS_3,WI_3,1941,0.000728,0.001214,0.000708


In [39]:
res.to_csv('../data/submission/Prophet_top_down_catboost_softmax.csv', index=False)